In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gensim
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
df_tweets = pd.read_csv('../input/Tweets.csv')
# Any results you write to the current directory are saved as output.

In [ ]:
df_tweets.head()

In [ ]:
list(df_tweets)

In [ ]:
df_tweets[1:10][
['airline_sentiment', 
 'airline_sentiment_confidence',
 'negativereason',
 'negativereason_confidence',
 'airline',
 'text']]

In [ ]:
with pd.option_context('display.max_colwidth', 250):
    for a in range(5):
        print(df_tweets[a:a + 1]['text'].to_string())

In [ ]:
df_tweets['tweet'] = df_tweets['text'].str.split()

In [ ]:
#Convert each tweet to an array
tweet_count = len(df_tweets['tweet'])
print("We have " + str(tweet_count) + " tweets.")

In [ ]:
model = gensim.models.Word2Vec( df_tweets['tweet'], size=50, window=10, min_count=2, workers=10)
model.save("Tweets")

In [ ]:
word = 'great'
vector = model.wv.most_similar(positive=word)

print(word)
print(vector)

In [ ]:
word = 'terrible'
vector = model.wv.most_similar(positive=word)

print(word)
print(vector)

In [ ]:
#Spacy Version
import spacy
from spacy.tokenizer import Tokenizer

nlp = spacy.load('en_core_web_lg') 
tokenizer = Tokenizer(nlp.vocab)

In [ ]:
df_model = df_tweets.loc[df_tweets['airline_sentiment'] != 'neutral']
total_tweets = len(df_model)
print("Total tweets: ", total_tweets )

positive_tweets = len(df_tweets['airline_sentiment'].loc[df_tweets['airline_sentiment'] == 'positive'])
negative_tweets = total_tweets - positive_tweets

print("Positive tweets: ", positive_tweets, '  ', round(100 * positive_tweets / total_tweets,2), '%' )
print("Negative tweets: ", negative_tweets, '  ', round(100 * negative_tweets / total_tweets,3), '%' )


df_model['target'] = pd.get_dummies(df_model['airline_sentiment'])['positive']

df_model = df_model.reset_index()

In [ ]:

df_length = len(df_model)
inputs = []
for n_step in range(0, df_length ):
    if n_step %1000 == 0:
        with pd.option_context('display.max_colwidth', 250):
            print(df_model['text'][n_step:n_step+1])
    tokens = nlp(str(df_model['text'][n_step:n_step+1]))
    inputs.append(tokens.vector)
    
df_vect = pd.DataFrame(inputs)


In [ ]:
df_vect['text'] = df_model['text']
df_vect['target'] = df_model['target']

In [ ]:
df_vect.head()

In [ ]:
print(len(df_vect.loc[df_vect['target'] == 0]))
print(len(df_vect.loc[df_vect['target'] == 1]))

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

feat_list = list(df_vect)[:-2]

forest = RandomForestClassifier()

vals = cross_val_score(forest, df_vect[feat_list], df_vect['target'], cv = 5 )
print(vals)
print(sum(vals)/len(vals))

In [ ]:
print("Positive tweets: ", positive_tweets, '  ', round(100 * positive_tweets / total_tweets,2), '%' )
print("Negative tweets: ", negative_tweets, '  ', round(100 * negative_tweets / total_tweets,3), '%' )

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_vect[feat_list], df_vect['target'], test_size=0.2)

print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)

In [ ]:
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split


sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())
X_test_res, y_test_res = sm.fit_sample(X_test, y_test.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

# Before OverSampling, counts of label '1': [345]

In [ ]:
print("Original: ")
print(y_train.value_counts())

total_data = len(y_train_res)
percent_positive = sum(y_train_res)/ total_data

print('\n\nNew Data:')
print('Data Points: ', total_data)
print('\nPositive Tweets: ', int(total_data * percent_positive), '   ', 100 * percent_positive, '%')
print('Negative Tweets: ', int(total_data * (1- percent_positive)), '   ', 100 * (1-percent_positive), '%')

Build a Random Forrest Model with the balanced data.

In [ ]:
forest = RandomForestClassifier(n_estimators=140, max_depth=4)
forest.fit(X_train_res, y_train_res)
print(round(100 * forest.score(X_test_res, y_test_res), 2), '%')

In [ ]:
import time
t0 = time.time()
gb_model = GradientBoostingClassifier( learning_rate=0.1, n_estimators=100)
gb_model.fit(X_train_res, y_train_res)
print(round(100 * gb_model.score(X_test_res, y_test_res), 2), '%')
print(round(time.time() - t0, 2), 'seconds')

In [ ]:
t0 = time.time()
gb_model1 = GradientBoostingClassifier( learning_rate=0.1, n_estimators=200)
gb_model1.fit(X_train_res, y_train_res)
print(round(100 * gb_model1.score(X_test_res, y_test_res), 2), '%')
print(round(time.time() - t0, 2), 'seconds')